### Chat History OakLang Graph
Here we'll be interacting with a server that's exposing a chat bot with memmory history being persisted on the backend.

In [1]:
from langserve import RemoteRunnable

cheese = RemoteRunnable("http://localhost:8000/cheese")

### Quick Graph Test

In [2]:
# define the thread_id
config = {"configurable": {"thread_id": "001"}}

In [ ]:
# Define the input and invoke
user_input = "Ey, qué tal?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
# Define the input and invoke
user_input = "Puedes resumirme los últimos 25 mensajes"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
from langgraph.types import Command
for event in cheese.stream(
    # provide value
    Command(resume={"action": "continue"}),
    config,
    stream_mode="updates",
):
    print(event)
    print("\n")

### Live Chat Cheese-Chatter Graph

In [ ]:
## Live chat via stream
# define the thread_id
config = {"configurable": {"thread_id": "002"}}

while True:
    user_input = input("Human: ")
    message_input = {"messages": [{"role": "human", "content": user_input}]}
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Cheese-Chatter: Adiós!")
        break
    else:
        print(f"Human: {user_input}") # to see the user input in the live chat
    for event in cheese.stream(message_input, config): 
        for key, value in event.items():
            if key == 'cheeseagent' and len(value["messages"][-1].content) > 0:
                print("Cheese-Chatter:", value["messages"][-1].content)     

In [ ]:
## Follow the nodes via stream
# define the thread_id
config = {"configurable": {"thread_id": "002"}}

# define the input
user_input = "Do you know what is an atom?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config):
    # stream() yields dictionaries with output keyed by node name
    for key, value in event.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

### Request API

In [ ]:
import requests

In [ ]:
response = requests.post(
    url='http://localhost:8000/cheese/invoke',
    json={
        'input': {
            "messages": [("human", "Hola, cómo estás")],
        },
        'config': {
            'configurable': {
                "thread_id": "004"
            }
        }
    }
)

print(response.json()["output"]["messages"][-1])

In [ ]:
## Test API
n_messages=25

url = f'http://***:8083/api/messages?size={n_messages}'
response = requests.get(url)

if response.status_code != 200:
    raise ValueError(f"Error: {n_messages} is not a valid argument")

data = response.json()

# Filter the id and timestand and reverse the messages
parsed_data = [{k: v for k, v in item.items() if k != 'timestamp' and k != 'id'} for item in reversed(data)]

parsed_data

## TEST HUMAN REVIEW

### TEST CUSTOM GRAPH

In [ ]:
from cheese.entity.statehandler import StateCommander
from typing_extensions import Literal, Any
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt
from langchain_openai import AzureChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode

@tool
def weather_search(city: str):
    """Search for the weather"""
    print("----")
    print(f"Searching for: {city}")
    print("----")
    return "Sunny!"


model = ChatOllama(model='llama3.1', temperature=0).bind_tools([weather_search])

class State(MessagesState):
    """Simple state."""


def call_llm(state):
    return {"messages": [model.invoke(state["messages"])]}

class CustomClass(StateCommander):
    @staticmethod
    def command(state:StateGraph(state_schema=Any)) -> Command[Literal['tool_node', 'call_llm']]: # type: ignore
        last_message = state["messages"][-1]
        tool_call = last_message.tool_calls[-1]

        # this is the value we'll be providing via Command(resume=<human_review>)
        human_review = interrupt(
            {
                "question": "Is this correct?",
                # Surface tool calls for review
                "tool_call": tool_call,
            }
        )

        review_action = human_review["action"]
        review_data = human_review.get("data")

        # if approved, call the tool
        if review_action == "continue":
            return Command(goto='tool_node')

        # update the AI message AND call tools
        elif review_action == "update":
            updated_message = {
                "role": "ai",
                "content": last_message.content,
                "tool_calls": [
                    {
                        "id": tool_call["id"],
                        "name": tool_call["name"],
                        # This the update provided by the human
                        "args": review_data,
                    }
                ],
                # This is important - this needs to be the same as the message you replacing!
                # Otherwise, it will show up as a separate message
                "id": last_message.id,
            }
            return Command(goto='tool_node', update={"messages": [updated_message]})

        # provide feedback to LLM
        elif review_action == "feedback":
            # NOTE: we're adding feedback message as a ToolMessage
            # to preserve the correct order in the message history
            # (AI messages with tool calls need to be followed by tool call messages)
            tool_message = {
                "role": "tool",
                # This is our natural language feedback
                "content": review_data,
                "name": tool_call["name"],
                "tool_call_id": tool_call["id"],
            }
            return Command(goto='call_llm', update={"messages": [tool_message]})


def route_after_llm(state) -> Literal['end', 'review']:
    if len(state["messages"][-1].tool_calls) == 0:
        return "end"
    else:
        return "review"



builder = StateGraph(State)
builder.add_node("call_llm", call_llm)
builder.add_node("tool_node", ToolNode(tools=[weather_search]))
builder.add_node("human_review_node", CustomClass.command)
builder.add_edge(START, "call_llm")
builder.add_conditional_edges("call_llm", route_after_llm, {
                                  "end": END, # If `tools not needed`, then we call the tool node.
                                  "review": "human_review_node", # Otherwise we finish.
                                  })
builder.add_edge("tool_node", "call_llm")

# Set up memory
memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

# View
display(Image(graph.get_graph().draw_mermaid_png()))

### TEST GRAPH HUMAN LOOP

In [1]:
## From LangServer
from langgraph.types import Command
from langserve import RemoteRunnable

graph = RemoteRunnable("http://localhost:8000/cheese")

In [1]:
## From LangGraph
from langgraph.types import Command
from cheese.workflow_builder import WorkflowBuilder
from cheese.config.config_graph import ConfigGraph

## Workflow Configuration
workflow_builder = WorkflowBuilder(config=ConfigGraph)
graph = workflow_builder.compile() # compile the graph

In [ ]:
# Input
initial_input = {"messages": [{"role": "human", "content": "cómo te encuentras?"}]}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
# Input
# initial_input = {"messages": [{"role": "human", "content": "qué tiempo hace en sf?"}]}
initial_input = {"messages": [{"role": "human", "content": "quieron un resumen 5000 mensajes"}]}

# Thread
thread = {"configurable": {"thread_id": "2"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
# Let's now continue executing from here
for event in graph.stream(
    # provide our natural language feedback!
    Command(
        resume={
            "action": "feedback",
            "data": "Perdona, quise decir 5 mensajes",
        }
    ),
    thread,
    stream_mode="updates"
):
    print(event)
    print("\n")

In [ ]:
for event in graph.stream(
    # provide value
    Command(resume={"action": "continue"}),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

### TEST NO STREAM

In [ ]:
# Input
initial_input = {"messages": [{"role": "human", "content": "cómo te encuentras?"}]}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
graph.invoke(initial_input, thread, stream_mode="updates")

In [ ]:
# Input
initial_input = {"messages": [{"role": "human", "content": "Quiero un resumen de 5000 mensajes."}]}

# Thread
thread = {"configurable": {"thread_id": "2"}}

# Run the graph until the first interruption
graph.invoke(initial_input, thread, stream_mode="updates")

In [ ]:
# Let's now continue executing from here
graph.invoke(
    # provide our natural language feedback!
    Command(
        resume={
            "action": "feedback",
            "data": "Perdona, quise decir 5 mensajes",
        }
    ),
    thread,
    stream_mode="updates",
)

In [ ]:
graph.invoke(
    # provide value
    Command(resume={"action": "continue"}),
    thread,
    stream_mode="updates",
)